In [ ]:
import torch, detectron2
#!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random
import cv2

In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
Data_Resister_training="app_train";
Data_Resister_valid="app_valid";
from detectron2.data.datasets import register_coco_instances
#dataDir=Path('./dataset')

register_coco_instances(Data_Resister_training,{}, './dataset/train/train.json', "./dataset/train")
register_coco_instances(Data_Resister_valid,{},'./dataset/val/val.json', "./dataset/val")

MetadataCatalog.get(Data_Resister_training).set(thing_classes=["Title","Menu","Ribbon"])
MetadataCatalog.get(Data_Resister_valid).set(thing_classes=["Title","Menu","Ribbon"])
metadata = MetadataCatalog.get(Data_Resister_training)

dataset_train = DatasetCatalog.get(Data_Resister_training)
dataset_valid = DatasetCatalog.get(Data_Resister_valid)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file("./output/model_config.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS='./output/model_final.pth'
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

In [ ]:
dataset_valid = DatasetCatalog.get(Data_Resister_valid)
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
for d in random.sample(dataset_valid, 3): 
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    print(outputs["instances"])
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    print(d["file_name"])
    
    plt.rcParams["figure.figsize"] = (20,6)
    plt.imshow(out.get_image()[:, :, ::-1])

    plt.show()

